In [1]:
!pip install deap

  Obtaining dependency information for deap from https://files.pythonhosted.org/packages/b2/98/537dadfab160eb7c39da979f52018b6d416e1d8fc86895cf5638e4538a12/deap-1.4.3-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/109.7 kB ? eta -:--:--
   ---------------------- ----------------- 61.4/109.7 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 109.7/109.7 kB 2.2 MB/s eta 0:00:00


In [12]:
import random
from deap import base, creator, tools, algorithms

In [13]:
# -----------------------------
# Problem definition
# -----------------------------

# List of tasks (jobs, activities, operations – depends on the story)
TASKS = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']

In [14]:
# Precedence constraints:
# key = task
# value = list of tasks that must be completed BEFORE this task
PRECEDENCE = {
    'C': ['A'],
    'D': ['A'],
    'E': ['B'],
    'F': ['C', 'D'],
    'G': ['E', 'F'],
    'H': ['G'],
    'I': ['F'],
    'J': ['H', 'I']
}

In [15]:
# -----------------------------
# Encoding: map tasks to integers
# -----------------------------

# Map each task to a unique integer
task2i = {t: i for i, t in enumerate(TASKS)}

# Reverse mapping: integer back to task label
i2task = {i: t for t, i in task2i.items()}

In [16]:
# Convert an integer chromosome into task labels
def decode(individual_ints):
    return [i2task[i] for i in individual_ints]

In [17]:
# Check whether a decoded task order respects precedence constraints
def is_valid(order_tasks):
    # position of each task in the order
    pos = {t: i for i, t in enumerate(order_tasks)}

    # for each precedence rule
    for t, preds in PRECEDENCE.items():
        for p in preds:
            # predecessor appears after the task → violation
            if pos[p] > pos[t]:
                return False
    return True

In [18]:
# -----------------------------
# Fitness function
# -----------------------------

def evaluate(individual_ints, w_violation=1000, w_gap=1):
    # Convert integer chromosome into task order
    order = decode(individual_ints)
    # Position of each task in the order
    pos = {t: i for i, t in enumerate(order)}
    violations = 0      # number of precedence violations
    gap_penalty = 0     # how far tasks are separated
    for t, preds in PRECEDENCE.items():
        for p in preds:
            if pos[p] > pos[t]:
                violations += 1
            else:
                # number of tasks between predecessor and task
                gap_penalty += (pos[t] - pos[p] - 1)
    # Final fitness value
    total = w_violation * violations + w_gap * gap_penalty
    return (total,)


In [19]:
# -----------------------------
# DEAP object creation
# -----------------------------
# Create a minimization fitness (negative weight)
try:
    creator.FitnessMin
except AttributeError:
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

In [20]:
# Individual = chromosome = list of genes
try:
    creator.Individual
except AttributeError:
    creator.create("Individual", list, fitness=creator.FitnessMin)

In [21]:
# -----------------------------
# Toolbox: GA operators
# -----------------------------
toolbox = base.Toolbox()

In [22]:
N = len(TASKS)

In [23]:
toolbox.register(
    "individual",
    tools.initIterate,
    creator.Individual,
    lambda: random.sample(range(N), N)
)

In [24]:
toolbox.register(
    "population",
    tools.initRepeat,
    list,
    toolbox.individual
)

In [25]:
toolbox.register("evaluate", evaluate)

In [26]:
toolbox.register("select", tools.selTournament, tournsize=3)

In [27]:
toolbox.register("mate", tools.cxOrdered)

In [28]:
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.2)

In [29]:
# -----------------------------
# GA execution
# -----------------------------

def run(seed=42, pop_size=80, ngen=200, cxpb=0.8, mutpb=0.2):
    random.seed(seed)
    pop = toolbox.population(n=pop_size)
    stats = tools.Statistics(lambda ind: ind.fitness.values[0])
    stats.register("min", min)
    stats.register("avg", lambda xs: sum(xs)/len(xs))
    hof = tools.HallOfFame(1)
    pop, log = algorithms.eaSimple(
        pop, toolbox,
        cxpb=cxpb, mutpb=mutpb, ngen=ngen,
        stats=stats, halloffame=hof, verbose=True
    )
    best = hof[0]
    best_order = decode(best)
    print("\nBest (ints):", best)
    print("Best (tasks):", best_order)
    print("Penalty:", best.fitness.values[0])
    print("Valid:", is_valid(best_order))
    return best, log

In [30]:
# Run the GA
best, log = run(seed=42)

gen	nevals	min 	avg    
0  	80    	2013	5564.57
1  	65    	1016	4690.79
2  	72    	2008	4240.59
3  	64    	1015	3977.26
4  	71    	1015	4226.38
5  	65    	2007	4063.1 
6  	71    	2007	4149.89
7  	76    	1012	3887.64
8  	72    	12  	3599.34
9  	74    	12  	3324   
10 	63    	12  	2786.16
11 	57    	11  	1849.72
12 	67    	11  	1387.69
13 	71    	11  	962.325
14 	68    	11  	637.788
15 	62    	11  	912.513
16 	67    	10  	599.6  
17 	64    	10  	686.938
18 	67    	10  	548.962
19 	72    	10  	611.237
20 	65    	10  	961.725
21 	66    	9   	811.663
22 	68    	9   	648.45 
23 	71    	9   	498.238
24 	68    	9   	372.75 
25 	72    	9   	1098.11
26 	65    	9   	873.55 
27 	66    	9   	660.025
28 	65    	9   	397.15 
29 	66    	9   	822.7  
30 	70    	9   	484.85 
31 	74    	9   	709.95 
32 	72    	9   	572.138
33 	68    	9   	672.888
34 	68    	9   	834.875
35 	66    	9   	673.125
36 	70    	9   	659.837
37 	69    	9   	535.112
38 	70    	9   	535.075
39 	68    	9   	785.138
40 	71    	9   	